In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
import requests
import json

In [9]:
api_key = '5S5T04QLSJIIMSU7'
symbol = 'AAPL'

url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}'

response = requests.get(url=url)
data = json.loads(response.text)

In [18]:
data

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-04-14',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2023-04-14': {'1. open': '164.59',
   '2. high': '166.32',
   '3. low': '163.82',
   '4. close': '165.21',
   '5. adjusted close': '165.21',
   '6. volume': '49386480',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-04-13': {'1. open': '161.63',
   '2. high': '165.8',
   '3. low': '161.42',
   '4. close': '165.56',
   '5. adjusted close': '165.56',
   '6. volume': '68445649',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-04-12': {'1. open': '161.22',
   '2. high': '162.06',
   '3. low': '159.78',
   '4. close': '160.1',
   '5. adjusted close': '160.1',
   '6. volume': '50133062',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-04-11': {'1. open': '162.35',
   '2.

In [22]:
daily_closing_prices = []
daily_opening_prices = []
for date in data['Time Series (Daily)']:
    daily_closing_prices.append(float(data['Time Series (Daily)'][date]['1. open']))
    daily_opening_prices.append(float(data['Time Series (Daily)'][date]['4. close']))

In [23]:
print(f"Opening Prices: {daily_opening_prices}")
print(f"Closing Prices: {daily_closing_prices}")

Opening Prices: [165.21, 165.56, 160.1, 160.8, 162.03, 164.66, 163.76, 165.63, 166.17, 164.9, 162.36, 160.77, 157.65, 158.28, 160.25, 158.93, 157.83, 159.28, 157.4, 155.0, 155.85, 152.99, 152.59, 150.47, 148.5, 150.59, 152.87, 151.6, 153.83, 151.03, 145.91, 145.31, 147.41, 147.92, 146.71, 149.4, 148.91, 148.48, 152.55, 153.71, 155.33, 153.2, 153.85, 151.01, 150.87, 151.92, 154.65, 151.73, 154.5, 150.82, 145.43, 144.29, 143.0, 145.93, 143.96, 141.86, 142.53, 141.11, 137.87, 135.27, 135.21, 135.94, 134.76, 133.41, 133.49, 130.73, 130.15, 129.62, 125.02, 126.36, 125.07, 129.93, 129.61, 126.04, 130.03, 131.86, 132.23, 135.45, 132.3, 132.37, 134.51, 136.5, 143.21, 145.47, 144.49, 142.16, 142.65, 140.94, 142.91, 146.63, 147.81, 148.31, 148.03, 141.17, 144.22, 148.11, 151.07, 150.18, 148.01, 151.29]
Closing Prices: [164.59, 161.63, 161.22, 162.35, 161.42, 162.43, 164.74, 166.595, 164.27, 162.44, 161.53, 159.37, 157.97, 159.94, 158.86, 158.83, 159.3, 157.32, 155.07, 156.08, 152.16, 151.19, 151

In [24]:
print(len(daily_opening_prices) == len(daily_closing_prices))

True


In [25]:
from sklearn.linear_model import LinearRegression

In [26]:
data = pd.DataFrame.from_dict(response.json()['Time Series (Daily)'], orient='index')

In [27]:
data = data.rename(columns={
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. adjusted close': 'Adjusted Close',
    '6. volume': 'Volume',
    '7. dividend amount': 'Dividend Amount',
    '8. split coefficient': 'Split Coefficient'
})

In [68]:
data = data.astype(float)
data.dropna()
print(data.head())

              Open    High     Low   Close  Adjusted Close      Volume  \
2023-04-14  164.59  166.32  163.82  165.21          165.21  49386480.0   
2023-04-13  161.63  165.80  161.42  165.56          165.56  68445649.0   
2023-04-12  161.22  162.06  159.78  160.10          160.10  50133062.0   
2023-04-11  162.35  162.36  160.51  160.80          160.80  47644217.0   
2023-04-10  161.42  162.03  160.08  162.03          162.03  47716882.0   

            Target  Price Change  Range  MA  
2023-04-14  165.56          0.62   2.50 NaN  
2023-04-13  160.10          3.93   4.38 NaN  
2023-04-12  160.80         -1.12   2.28 NaN  
2023-04-11  162.03         -1.55   1.85 NaN  
2023-04-10  164.66          0.61   1.95 NaN  


In [69]:
data['Target'] = data['Close'].shift(-1)

# Daily price change
data['Price Change'] = data['Close'] - data['Open']
data['Range'] = data['High'] - data['Low']

# Moving averages
n = 7
data['MA'] = data['Close'].rolling(n).mean()

In [70]:
data.head()

,Open,High,Low,Close,Adjusted Close,Volume,Target,Price Change,Range,MA
2023-04-14,164.59,166.32,163.82,165.21,165.21,49386480.0,165.56,0.62,2.50,NaN
2023-04-13,161.63,165.80,161.42,165.56,165.56,68445649.0,160.10,3.93,4.38,NaN
2023-04-12,161.22,162.06,159.78,160.10,160.10,50133062.0,160.80,-1.12,2.28,NaN
2023-04-11,162.35,162.36,160.51,160.80,160.80,47644217.0,162.03,-1.55,1.85,NaN
2023-04-10,161.42,162.03,160.08,162.03,162.03,47716882.0,164.66,0.61,1.95,NaN


In [71]:
data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adjusted Close', 'Volume', 'Target',
       'Price Change', 'Range', 'MA'],
      dtype='object')

In [72]:
X = data.drop(['Target'], axis=1)
y = data['Target']

In [73]:
train_size = int(0.8 * len(X))

X_train, X_test = X[: train_size], X[train_size:]
y_train, y_test = y[: train_size], y[train_size:]

In [74]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# y_train = imputer.fit_transform(y_train)
# y_test = imputer.fit_transform(y_test)

In [75]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [76]:
y_predict = lr.predict(X_test)